In [48]:
import sympy

def get_M(a, b, b0):
    X = a-b
    X = X / sympy.sqrt( X.dot(X) )
    Z = X.cross(b0-b)
    Z = Z / sympy.sqrt( Z.dot(Z) )
    Y = Z.cross(X)
    M=sympy.Matrix([[X[0],Y[0],Z[0]],[X[1],Y[1],Z[1]],[X[2],Y[2],Z[2]]])
    return sympy.simplify(M)

def get_W(dist, ca,ct,sa,st):
    W = sympy.Matrix([
         dist * ca,
         dist * sa * ct,
         dist * sa * st
    ])
    return W

def build_acc_water( a, b, b0, dist, ca,ct,sa,st ):
    M=get_M(a, b, b0)
    wat_loc = get_W(dist, ca,ct,sa,st)
    water = M*wat_loc + a
    return water

ax,ay,az = sympy.symbols('a0,a1,a2')
A = sympy.Matrix([ax,ay,az])
bx,by,bz = sympy.symbols('b0,b1,b2')
B = sympy.Matrix([bx,by,bz])
b0x,b0y,b0z = sympy.symbols('c0,c1,c2')
B0 = sympy.Matrix([b0x,b0y,b0z])
d,ca,ct,sa,st = sympy.symbols('dist,cos_a,cos_t,sin_a,sin_t') # cos_a=cos(pi-a)
sympy.init_printing(False)

acc_wat = build_acc_water(A,B,B0,d,ca,ct,sa,st)

In [99]:
a_m_b,b0_m_b,axb_x,axb_y,axb_z = sympy.symbols('a_m_b,b0_m_b,axb_x,axb_y,axb_z')

Derivs={}
for s in ('a0','a1','a2','b0','b1','b2','c0','c1','c2'):
    Derivs[s] = sympy.diff(acc_wat,s)
    Derivs[s] = Derivs[s].subs(sympy.sqrt((ax-bx)**2+(ay-by)**2+(az-bz)**2), a_m_b)
    Derivs[s] = Derivs[s].subs(sympy.sqrt((b0x-bx)**2+(b0y-by)**2+(b0z-bz)**2), b0_m_b)
    Derivs[s] = Derivs[s].subs((ax - bx)*(by - b0y) - (ay - by)*(bx - b0x), axb_x)
    Derivs[s] = Derivs[s].subs((ax - bx)*(bz - b0z) - (az - bz)*(bx - b0x), axb_y)
    Derivs[s] = Derivs[s].subs((ay - by)*(bz - b0z) - (az - bz)*(by - b0y), axb_z)
    Derivs[s] = Derivs[s].subs(axb_x**2 +axb_y**2+axb_z**2, 'axb_xyz2')

In [103]:
ds=[ Derivs[s] for s in ('a0','a1','a2','b0','b1','b2','c0','c1','c2') ]
ds

[Matrix([
[                                         -a_m_b*axb_z*dist*sin_a*sin_t*(-(axb_x*(2*b1 - 2*c1) - axb_y*(-2*b2 + 2*c2))/(2*a_m_b**2) - axb_xyz2*(-2*a0 + 2*b0)/(2*a_m_b**4))/(axb_xyz2*sqrt(axb_xyz2/a_m_b**2)) + 1 + cos_t*dist*sin_a*(-(axb_x*(2*b1 - 2*c1) + axb_y*(2*b2 - 2*c2))/(2*a_m_b**2) - axb_xyz2*(-2*a0 + 2*b0)/(2*a_m_b**4))*(axb_x*(a1 - b1) + axb_y*(a2 - b2))/(axb_xyz2*sqrt(axb_xyz2/a_m_b**2)) + cos_a*dist/a_m_b + cos_t*dist*sin_a*((a1 - b1)*(b1 - c1) + (a2 - b2)*(b2 - c2))/(a_m_b**2*sqrt(axb_xyz2/a_m_b**2)) - axb_z*dist*sin_a*sin_t*(-a0 + b0)/(a_m_b**3*sqrt(axb_xyz2/a_m_b**2)) + cos_a*dist*(-a0 + b0)*(a0 - b0)/a_m_b**3 + cos_t*dist*sin_a*(-2*a0 + 2*b0)*(axb_x*(a1 - b1) + axb_y*(a2 - b2))/(a_m_b**4*sqrt(axb_xyz2/a_m_b**2))],
[              a_m_b*axb_y*dist*sin_a*sin_t*(-(axb_x*(2*b1 - 2*c1) - axb_y*(-2*b2 + 2*c2))/(2*a_m_b**2) - axb_xyz2*(-2*a0 + 2*b0)/(2*a_m_b**4))/(axb_xyz2*sqrt(axb_xyz2/a_m_b**2)) + cos_t*dist*sin_a*(-(axb_x*(2*b1 - 2*c1) + axb_y*(2*b2 - 2*c2))/(2*a_m_b

In [102]:
repl,red = sympy.cse(ds, symbols=sympy.utilities.iterables.numbered_symbols(start=101), ignore=(d,ca,ct,sa,st))
keysS = []
valsS = []
for key, val in repl:
    keysS.append(str(key))
    valsS.append(str(val))

lhsS = []
rhsS = []
for i in range(3):
    for j in range(3):
        for k in range(3):
            if (red[3*i+j][k]!=0):
                lhsS.append('dwda['+str(i)+','+str(j)+','+str(k)+']')
                rhsS.append(str(red[3*i+j][k]))

for i in range(len(keysS)):
    print (keysS[i],'=',valsS[i])
for i in range(len(lhsS)):
    print (lhsS[i],'=',rhsS[i])

x101 = -a0
x102 = b0 + x101
x103 = -b0
x104 = a0 + x103
x105 = x102*x104
x106 = a_m_b**(-3)
x107 = -b1
x108 = a1 + x107
x109 = -c1
x110 = b1 + x109
x111 = -b2
x112 = a2 + x111
x113 = -c2
x114 = b2 + x113
x115 = x112*x114
x116 = a_m_b**(-2)
x117 = 1/sqrt(axb_xyz2*x116)
x118 = 2*a0
x119 = -x118
x120 = 2*b0
x121 = x119 + x120
x122 = a_m_b**(-4)
x123 = x121*x122
x124 = axb_x*x108
x125 = axb_y*x112
x126 = x124 + x125
x127 = 1/axb_xyz2
x128 = axb_xyz2/2
x129 = -x123*x128
x130 = 2*b1
x131 = 2*c1
x132 = -x131
x133 = axb_x*(x130 + x132)
x134 = 2*b2
x135 = 2*c2
x136 = -x135
x137 = x134 + x136
x138 = x116/2
x139 = x127*(x129 - x138*(axb_y*x137 + x133))
x140 = axb_z*x127
x141 = -x134
x142 = x129 - x138*(-axb_y*(x135 + x141) + x133)
x143 = 1/a_m_b
x144 = -axb_x
x145 = axb_z*x112
x146 = axb_x*x104
x147 = x145 - x146
x148 = c1 + x107
x149 = axb_y*x104
x150 = axb_z*x108
x151 = x149 + x150
x152 = 2*x114
x153 = -c0
x154 = b0 + x153
x155 = -a1
x156 = b1 + x155
x157 = c0 + x103
x158 = 2*a1
x159 = -x158
x1

In [61]:
temp = ((ax - bx)**2).subs(ax - bx,'ab')
temp

ab**2

In [62]:
sympy.diff(temp,'a0'),

(0,)